## General Import and Setup

In [43]:
import pandas as pd
import plotly.express as px
import spacy
from collections import Counter
from nltk.corpus import stopwords
import string
from tabulate import tabulate
from textblob_de import TextBlobDE

In [42]:
nlp = spacy.load("de_core_news_sm")
data = pd.read_csv('../data/raw/dataset_cleaned_file_v5.csv')
data.head()

,genre,interpret,album,release_year,songtitle,decade,song_id,full_title,release_date_components_raw,release_date_for_display,stats_raw,lyrics,weekday,word_count
0,Pop,Adel Tawil,Lieder,2013,Immer da,2010,256740,Immer da by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}",2013-11-08,"{'unreviewed_annotations': 0, 'hot': False}",Manchmal hast du gar nichts mehr Nur noch de...,Friday,60
1,Pop,Adel Tawil,Lieder,2013,Wenn Du liebst,2010,256742,Wenn du liebst by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}",2013-11-08,"{'unreviewed_annotations': 4, 'hot': False, 'p...",Ganz allein im schwarzen Licht wie hinter Gla...,Friday,100
2,Pop,Adel Tawil,Lieder,2013,Lieder,2010,236655,Lieder by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}",2013-11-08,"{'unreviewed_annotations': 0, 'hot': False, 'p...",Ich ging wie ein Ägypter Hab' mit Tauben gew...,Friday,145
3,Pop,Adel Tawil,Lieder,2013,Weinen,2010,256745,Weinen by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}",2013-11-08,"{'unreviewed_annotations': 0, 'hot': False, 'p...","Du warst die ganze Welt für mich Warst das, ...",Friday,69
4,Pop,Adel Tawil,Lieder,2013,Unter Wasser,2010,256747,Unter Wasser by Adel Tawil,"{'year': 2013, 'month': 11, 'day': 8}",2013-11-08,"{'unreviewed_annotations': 0, 'hot': False}","Keine Chance, es ist viel zu tief Keine Chan...",Friday,108


### Checking the Vocabulary of the Artists

In [16]:
grouped_data = data.groupby("interpret")

tokenized_words = grouped_data["lyrics"].apply(lambda x: ' '.join(x).split())

unique_word_counts = tokenized_words.apply(lambda x: len(set(x)))

num_songs = grouped_data.size()


df = pd.DataFrame({'Interpret': unique_word_counts.index, 'Word Count': unique_word_counts, 'Number of Songs': num_songs})

fig = px.scatter(df, x='Word Count', y='Number of Songs', hover_data=['Interpret'], title='Number of Songs vs. Word Count')

fig.show()

### Checking the 10 most used words for each Genre

In [34]:
### Function for filtering stopwords and punctuation
def filter_stopwords(tokens):
    german_stopwords = set(stopwords.words("german"))
    filtered_tokens = []
    for token in tokens:
        if token.lower() not in german_stopwords and token not in string.punctuation:
            filtered_tokens.append(token.lower())
    return filtered_tokens

data["filtered_lyrics"] = data["lyrics"].apply(lambda x: filter_stopwords([token.text for token in nlp(x)]))
grouped_data = data.groupby("genre")

genre_word_counts = {}
for genre, group in grouped_data:
    lyrics = group["filtered_lyrics"].sum()
    word_counts = Counter(lyrics)
    genre_word_counts[genre] = word_counts


top_10_words_per_genre = {}
for genre, word_counts in genre_word_counts.items():
    top_words = word_counts.most_common(11) ### see down below
    top_10_words_per_genre[genre] = top_words

for genre, top_words in top_10_words_per_genre.items():
    print("Genre:", genre)
    df = pd.DataFrame(top_words, columns=["Word", "Frequency"])
    print(df)
    print()

Genre: Pop
     Word  Frequency
0               2783
1    mehr       1272
2   immer       1056
3   leben        766
4     mal        726
5   schon        682
6    zeit        677
7     nie        657
8    weiß        606
9    welt        573
10   geht        570

Genre: Rap
     Word  Frequency
0               7184
1     mal       2149
2    mehr       1983
3    yeah       1679
4   immer       1677
5   leben       1643
6      ja       1564
7   schon       1439
8    komm       1424
9    geht       1392
10   mach       1290

Genre: Rock
     Word  Frequency
0               3631
1    mehr       1129
2   immer       1079
3     mal        815
4   schon        730
5   leben        628
6    hast        611
7    weiß        594
8      ja        553
9    welt        550
10    nie        533

Genre: Schlager
     Word  Frequency
0                876
1   nacht        632
2    mehr        614
3     nie        510
4   liebe        488
5   immer        432
6    ganz        386
7   schon        386
8 

In [35]:
###if token.lower() not in german_stopwords and token not in string.punctuation and token.strip() != "":
###            filtered_tokens.append(token.lower())
### takes super long to run >=2hours... probably bugged somehow ?
### used a fix in counting the top 11 words and just excluding the first one, since that is whitespace

top_10_words_per_genre = {}
for genre, word_counts in genre_word_counts.items():
    top_words = word_counts.most_common(11)
    top_10_words_per_genre[genre] = top_words[1:]  # Exclude the first item

for genre, top_words in top_10_words_per_genre.items():
    print("Genre:", genre)
    df = pd.DataFrame(top_words, columns=["Word", "Frequency"])
    print(df)
    print()

Genre: Pop
    Word  Frequency
0   mehr       1272
1  immer       1056
2  leben        766
3    mal        726
4  schon        682
5   zeit        677
6    nie        657
7   weiß        606
8   welt        573
9   geht        570

Genre: Rap
    Word  Frequency
0    mal       2149
1   mehr       1983
2   yeah       1679
3  immer       1677
4  leben       1643
5     ja       1564
6  schon       1439
7   komm       1424
8   geht       1392
9   mach       1290

Genre: Rock
    Word  Frequency
0   mehr       1129
1  immer       1079
2    mal        815
3  schon        730
4  leben        628
5   hast        611
6   weiß        594
7     ja        553
8   welt        550
9    nie        533

Genre: Schlager
    Word  Frequency
0  nacht        632
1   mehr        614
2    nie        510
3  liebe        488
4  immer        432
5   ganz        386
6  schon        386
7  leben        378
8   herz        356
9   lass        352



In [40]:
### Example on how to plot the data, couldnt really find anything prettier (didnt search that long tho)
table_data = []
headers = ["Word"] + list(top_10_words_per_genre.keys())
for i in range(10):
    row_data = [top_10_words_per_genre[genre][i] for genre in top_10_words_per_genre]
    table_data.append(row_data)

table = tabulate(table_data, headers, tablefmt="pretty")
print(table)

+-----------------+-----------------+-----------------+----------------+
|      Word       |       Pop       |       Rap       |      Rock      |
+-----------------+-----------------+-----------------+----------------+
| ('mehr', 1272)  |  ('mal', 2149)  | ('mehr', 1129)  | ('nacht', 632) |
| ('immer', 1056) | ('mehr', 1983)  | ('immer', 1079) | ('mehr', 614)  |
| ('leben', 766)  | ('yeah', 1679)  |  ('mal', 815)   |  ('nie', 510)  |
|  ('mal', 726)   | ('immer', 1677) | ('schon', 730)  | ('liebe', 488) |
| ('schon', 682)  | ('leben', 1643) | ('leben', 628)  | ('immer', 432) |
|  ('zeit', 677)  |  ('ja', 1564)   |  ('hast', 611)  | ('ganz', 386)  |
|  ('nie', 657)   | ('schon', 1439) |  ('weiß', 594)  | ('schon', 386) |
|  ('weiß', 606)  | ('komm', 1424)  |   ('ja', 553)   | ('leben', 378) |
|  ('welt', 573)  | ('geht', 1392)  |  ('welt', 550)  | ('herz', 356)  |
|  ('geht', 570)  | ('mach', 1290)  |  ('nie', 533)   | ('lass', 352)  |
+-----------------+-----------------+--------------

### Sentiment Analysis of a Genre

In [44]:
genre_data = data[data["genre"] == "Rap"]

sentiments = []
for lyric in genre_data["lyrics"]:
    blob = TextBlobDE(lyric)
    sentiment = blob.sentiment
    sentiments.append((lyric, sentiment.polarity, sentiment.subjectivity))

sentiments_df = pd.DataFrame(sentiments, columns=["Lyric", "Polarity", "Subjectivity"])
print(sentiments_df.head())

                                               Lyric  Polarity  Subjectivity
0   Okay, Ansage, Freundchen, ich warne dich Ey, ...  0.067500      0.100000
1    Ey Leute, passt doch auf, ihr lebt in Gefahr...  0.115972      0.006944
2   Okay, Ellen lang und elegant Keule Mutterfick... -0.031250      0.015625
3   Hallihallöchen, ist lange her Schatz Ich vers...  0.048352      0.000000
4    Ich steh auf am Kleister riechen und gleiche... -0.054167      0.144444


In [47]:
pop_data = data[data["genre"] == "Pop"]
rap_data = data[data["genre"] == "Rap"]
pop_lyrics_sentiments = []
rap_lyrics_sentiments = []


pop_sentiments = []
for lyric in pop_data["lyrics"]:
    blob = TextBlobDE(lyric)
    sentiment = blob.sentiment.polarity
    pop_sentiments.append(sentiment)
    pop_lyrics_sentiments.append((lyric, sentiment))

rap_sentiments = []
for lyric in rap_data["lyrics"]:
    blob = TextBlobDE(lyric)
    sentiment = blob.sentiment.polarity
    rap_sentiments.append(sentiment)
    rap_lyrics_sentiments.append((lyric, sentiment))

pop_avg_polarity = sum(pop_sentiments) / len(pop_sentiments)
rap_avg_polarity = sum(rap_sentiments) / len(rap_sentiments)

pop_lyrics_sentiments.sort(key=lambda x: x[1], reverse=True)
rap_lyrics_sentiments.sort(key=lambda x: x[1], reverse=True)

most_positive_pop_lyric = pop_lyrics_sentiments[0][0]
most_negative_pop_lyric = pop_lyrics_sentiments[-1][0]
most_positive_rap_lyric = rap_lyrics_sentiments[0][0]
most_negative_rap_lyric = rap_lyrics_sentiments[-1][0]

print("Average Polarity - Pop:", pop_avg_polarity)
print("Average Polarity - Rap:", rap_avg_polarity)
print("Most Positive Pop Lyric:\n", most_positive_pop_lyric)
print("\nMost Negative Pop Lyric:\n", most_negative_pop_lyric)
print("\nMost Positive Rap Lyric:\n", most_positive_rap_lyric)
print("\nMost Negative Rap Lyric:\n", most_negative_rap_lyric)

Average Polarity - Pop: 0.13076579929096935
Average Polarity - Rap: 0.07712942875037107
Most Positive Pop Lyric:
   Ich schlafe ein Der nächste Schritt heißt Koma Das Fenster unten Und der Nachtwind rein Ich seh die Bäume Und ihre Wipfel so verschwommen Kann meinen Namen Grad so lallen Und es fährt mich nach Haus Es fährt mich wieder weit raus Und es fährt mich nach Haus Es fährt mich wieder weit raus  Ich sehe ein Paar Augen Höre vertraute Laute Kurzes Blaulicht, dann nur Mondschein Und eine Weite mit Dingen, die ich liebe Ich wünschte, es könnte immer so sein  Und es fährt mich nach Haus Es fährt mich wieder weit raus Und es fährt mich nach Haus Es fährt mich wieder weit raus Weit raus  Und es fährt mich nach Haus Es fährt mich wieder weit raus Und es fährt mich nach Haus Es fährt mich wieder weit raus

Most Negative Pop Lyric:
 Ich würd' für dich jeden Boxer verschlagen Ich würd' für dich alle Einkaufstüten tragen Ich würd' für dich wenn du willst ein wildes Tier Doch ich bin für di